# Quantum Perlin Noise


The quantum approach to Perlin noise generation involves representing noise as a quantum state. The height2state function converts a classical heightmap into a quantum state vector where the amplitudes of the state correspond to the heights. This state is normalized to ensure that the total probability is 1.

The quantum_tartan function prepares and executes a quantum circuit. It initializes the quantum state, applies a rotation gate parameterized by theta, and then measures the quantum state. The results, whether from a simulator or actual quantum hardware, are used to determine the noise characteristics. The measurement outcomes are then converted back into a heightmap using the counts2height function, which normalizes and possibly applies logarithmic transformation to the heights.

As we can see, quantum computing adds another level of complexity into the terrain generation using Perlin noise. Classical approach is still more effecient, faster and simpler to implement. However, the purpose of the original paper was to show that quantum computing can be used in Game Development even in its current state of development when it doesn't outperform classical computers.

The original work and code: https://medium.com/qiskit/creating-infinite-worlds-with-quantum-computing-5e998e6d21c2

I don't own anything here. My objective is to explain to people how code and the logic works, because I had a hard time going through this when I first started.


In [ ]:
from PIL import Image
from IPython.display import display

def height2image(Z, terrain=None):
    # Converts a heightmap Z into a PIL image
    # If terrain is None, creates a black and white image
    # Otherwise, uses terrain thresholds to create a colored image
    image = {}
    for pos in Z:
        if terrain:
            if Z[pos] < terrain[0]:
                image[pos] = (50, 120, 200)  # Sea
            elif Z[pos] < terrain[1]:
                image[pos] = (220, 220, 10)  # Beach
            elif Z[pos] < terrain[2]:
                image[pos] = (100, 200, 0)   # Grass
            elif Z[pos] < terrain[3]:
                image[pos] = (75, 150, 0)    # Forest
            elif Z[pos] < terrain[4]:
                image[pos] = (200, 200, 200) # Mountain
            else:
                image[pos] = (255, 255, 255) # Snow
        else:
            z = int(255 * Z[pos])
            image[pos] = (z, z, z)  # Grayscale
    
    # Create an image from the heightmap
    X = max(Z.keys())[0] + 1
    Y = max(Z.keys())[1] + 1
    img = Image.new('RGB', (X, Y))
    for x in range(img.size[0]):
        for y in range(img.size[1]):
            img.load()[x, y] = image[x, y]
    return img

def plot_height(Z, terrain=[5/16, 6/16, 9/16, 12/16, 14/16], zoom=None):
    # Display the heightmap as an image
    # Uses terrain thresholds by default
    img = height2image(Z, terrain=terrain)
    if zoom:
        img = img.resize((zoom * img.size[0], zoom * img.size[1]), Image.LANCZOS)
    img.save('temp.png')
    display(Image.open('temp.png'))


In [ ]:
from opensimplex import OpenSimplex
import random

def simplex(L, period):
    # Create a heightmap for an L[0]xL[1] grid using simplex noise
    gen = OpenSimplex(seed=random.randint(0, 10**20))  # Initialize simplex noise generator with a random seed
    Z = {}
    for x in range(L[0]):
        for y in range(L[1]):
            # Scale coordinates to fit the period and center around 0
            xx = period[0] * (x / L[0] - 0.5)
            yy = period[1] * (y / L[1] - 0.5)
            # Generate noise value, normalize to [0, 1] range
            Z[x, y] = gen.noise2(xx, yy) / 2 + 0.5
    return Z

In [ ]:
n = 10
shots = 4**n

In [ ]:
import numpy as np

def get_L(n):
    # Determine the size of the grid based on the number of qubits
    Lx = int(2**np.ceil(n/2))
    Ly = int(2**np.floor(n/2))
    return [Lx, Ly]

def make_grid(n):
    # Create a grid where each point is assigned a unique n-bit string
    [Lx, Ly] = get_L(n)

    strings = {}
    for y in range(Ly):
        for x in range(Lx):
            strings[(x, y)] = ''

    # Generate n-bit strings based on the position in the grid
    for (x, y) in strings:
        for j in range(n):
            if (j % 2) == 0:
                xx = np.floor(x / 2**(j / 2))
                strings[(x, y)] = str(int((xx + np.floor(xx / 2)) % 2)) + strings[(x, y)]
            else:
                yy = np.floor(y / 2**((j - 1) / 2))
                strings[(x, y)] = str(int((yy + np.floor(yy / 2)) % 2)) + strings[(x, y)]

    # Adjust strings to center the grid around '0'*n
    center = '0' * n
    current_center = strings[(int(np.floor(Lx / 2)), int(np.floor(Ly / 2)))]
    diff = ''
    for j in range(n):
        diff += '0' * (current_center[j] == center[j]) + '1' * (current_center[j] != center[j])
    for (x, y) in strings:
        newstring = ''
        for j in range(n):
            newstring += strings[(x, y)][j] * (diff[j] == '0') + ('0' * (strings[(x, y)][j] == '1') + '1' * (strings[(x, y)][j] == '0')) * (diff[j] == '1')
        strings[(x, y)] = newstring

    # Create a grid dictionary mapping n-bit strings to positions
    grid = {}
    for y in range(Ly):
        for x in range(Lx):
            grid[strings[(x, y)]] = (x, y)

    return strings


In [ ]:
def normalize_height(Z):
    # Scales heights so that the maximum value is 1 and the minimum value is 0
    maxZ = max(Z.values())  # Find the maximum height value
    minZ = min(Z.values())  # Find the minimum height value
    for pos in Z:
        # Normalize each height value to the range [0, 1]
        Z[pos] = (Z[pos] - minZ) / (maxZ - minZ)
    return Z

def counts2height(counts, grid, log=False):
    # Converts counts to a heightmap based on grid positions, with optional logarithmic scaling and normalization
    Z = {}
    for pos in grid:
        # Set height based on counts value or default to 0 if not available
        Z[pos] = counts.get(grid[pos], 0)
    
    if log:
        for pos in Z:
            # Apply a small value to avoid log(0) and compute the logarithm base 2
            Z[pos] = max(Z[pos], 1 / len(grid) ** 2)
            Z[pos] = np.log(Z[pos]) / np.log(2)
    
    # Normalize the height values to the range [0, 1]
    Z = normalize_height(Z)    
    return Z


In [ ]:
def height2state(Z, grid):
    # Converts a heightmap (Z) into a quantum state vector
    N = len(grid)  # Total number of possible states (based on the size of the grid)
    state = [0] * N  # Initialize a quantum state vector with zeros

    for pos in Z:
        # Set the amplitude of the quantum state corresponding to the bit string of grid position 'pos'
        state[int(grid[pos], 2)] = np.sqrt(Z[pos])
    
    R = sum(np.absolute(state) ** 2)  # Compute the normalization factor (sum of squared amplitudes)
    # Normalize the quantum state vector to ensure it is a valid quantum state
    state = [amp / np.sqrt(R) for amp in state]
    
    return state


In [ ]:
def state2counts(state, shots=None):
    # Convert quantum state vector to counts of bit strings
    N = len(state)
    n = int(np.log2(N))  # Number of qubits

    if shots is None:
        shots = N**2  # Default number of shots

    counts = {}
    for j in range(N):
        # Convert index to binary string
        string = bin(j)[2:]
        string = '0' * (n - len(string)) + string
        
        # Calculate the probability and multiply by number of shots
        counts[string] = np.absolute(state[j])**2 * shots

    return counts


In [ ]:
L = get_L(n)
# Get the dimensions of the grid based on the number of qubits (n).
# L is a list [Lx, Ly] representing the width and height of the grid.

Z = simplex(L, [10, 10])
# Generate a heightmap Z for the grid using simplex noise.

grid = make_grid(n)
# Create a dictionary that maps grid coordinates to n-bit strings.
# These bit strings represent the binary encoding of the coordinates.

state = height2state(Z, grid)
# Convert the heightmap Z into a quantum state vector.
# Each entry in the quantum state vector corresponds to a bit string in the grid, with amplitudes derived from the heightmap values.


In [ ]:
counts = state2counts(state)
# Convert the quantum state vector into a dictionary of counts (probability distributions).
# Each entry represents the probability of measuring a specific bit string.

Z = counts2height(counts, grid)
# Convert the counts back into a heightmap Z using the grid mapping.
# The height values are scaled and normalized.

plot_height(Z, terrain=None, zoom=5)
# Visualize the heightmap Z.
# The image is displayed with a zoom factor of 5, without using any terrain thresholds.


In [ ]:
import ipywidgets as widgets
from ipywidgets import Checkbox, ToggleButton, Layout, HBox, VBox

def get_boxes(L, value=True):
    # Create a grid of ToggleButtons with size based on L and initial value.
    
    width = str(500 / L[0]) + 'px'  # Width of each button
    height = str(500 / L[1]) + 'px'  # Height of each button

    box = {}
    for y in range(L[1]):
        for x in range(L[0]):
            # Create a ToggleButton for each position in the grid
            box[x, y] = widgets.ToggleButton(value=value, button_style='', layout=Layout(width=width, height=height))
            
    return box


In [ ]:
box = get_boxes(L)

# Create a vertical box layout containing horizontal boxes of ToggleButtons
# Each horizontal box (HBox) contains a row of ToggleButtons
# The vertical box (VBox) stacks these horizontal boxes to form the grid
VBox([HBox([box[x, y] for x in range(L[0])]) for y in range(L[1])])


In [ ]:
def flat_height(L):
    # Create a height map where all values are initialized to 0
    Z = {}
    for x in range(L[0]):
        for y in range(L[1]):
            Z[x, y] = 0
    return Z

# Initialize the height map with zeros
Z = flat_height(L)

# Update the height map based on the values of the buttons
# Set height to 1 where the corresponding button value is False
for y in range(L[1]):
    for x in range(L[0]):
        if box[x, y].value == False:
            Z[x, y] = 1

# Plot the resulting height map with specified zoom level
plot_height(Z, terrain=None, zoom=5)


In [ ]:
from qiskit import *
import time
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import QiskitRuntimeService

def quantum_tartan(seed, theta, grid=None, shots=1, log=True):
    # Determine the number of qubits from the seed length
    n = int(np.log2(len(seed)))
        
    # Generate the grid if not provided
    if grid is None:
        grid = make_grid(n)

    # Convert the height map to a quantum state
    state = height2state(seed, grid)

    # Create a quantum circuit with 'n' qubits
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)
    qc.initialize(state, q)  # Initialize the quantum state
    qc.ry(2 * np.pi * theta, q)  # Apply rotation
    qc.save_statevector()  # Save the state vector
    
    # Choose backend based on the number of shots
    if shots > 1:
        try:
            service = QiskitRuntimeService()
            backend = service.least_busy(simulator=False, operational=True)  # Get the least busy backend
        except:
            print('An IBMQ account is required to use a real device\nSee https://github.com/Qiskit/qiskit-terra/blob/master/README.md')
    else:
        backend = AerSimulator(method='statevector')  # Use a local simulator if shots <= 1

    # Add measurement if shots > 1
    if shots > 1:
        c = ClassicalRegister(n)
        qc.add_register(c)
        qc.measure(q, c)
    
    # Execute the quantum circuit and measure the time taken
    start = time.time()
    print('Quantum job initiated on', backend.name)
    compiled_circuit = transpile(qc, backend)
    job = backend.run(compiled_circuit, shots=shots)
    end = time.time()
    print('Quantum job complete after', int(end-start), 'seconds')
    
    # Process results based on the number of shots
    if shots > 1:
        counts = job.result().get_counts()  # Get counts from the result
    else:
        counts = state2counts(job.result().get_statevector())  # Convert statevector to counts
        
    # Convert counts to height map and return
    Z = counts2height(counts, grid, log=log)   
    
    return Z, grid


In [ ]:
# Generate the height map and grid using the quantum_tartan function
Z, grid = quantum_tartan(Z, 0.01)

# Plot the resulting height map with a zoom level of 5
plot_height(Z, terrain=None, zoom=5)


In [ ]:
def shuffle_grid(grid):
    # Determine the number of bits in each string based on the length of the grid
    n = int(np.log(len(grid)) / np.log(2))
    
    # Create a list of indices from 0 to n-1 and shuffle them
    order = [j for j in range(n)]
    random.shuffle(order)
    
    # Create a new grid with the bit positions shuffled according to 'order'
    new_grid = {}
    for pos in grid:
        new_string = ''
        for j in order:
            new_string = grid[pos][j] + new_string
        new_grid[pos] = new_string
    
    return new_grid

def shuffle_height(Z, grid):
    # Shuffle the grid and update the height map Z according to the new grid mapping
    new_grid = shuffle_grid(grid)
    new_Z = {}
    for pos in Z:
        string = grid[pos] 
        new_pos = list(new_grid.keys())[list(new_grid.values()).index(string)]
        new_Z[new_pos] = Z[pos]
        
    return new_Z, new_grid

# Shuffle the grid and update the height map Z accordingly
Z, grid = shuffle_height(Z, grid)

# Plot the shuffled height map with a zoom level of 5
plot_height(Z, terrain=None, zoom=5)


In [ ]:
def rotate_height(Z, theta):
    # Rotate the heightmap Z by an angle theta (in fractions of π)
    
    # Get the dimensions of the original heightmap
    L = list(max(Z))
    # Calculate the midpoint of the original heightmap dimensions
    mid = [(L[j] + 1) / 2 for j in range(2)]
    
    # Define new dimensions for the rotated heightmap
    Lr = [int(1.6 * (L[j] + 1)) for j in range(2)]
    # Calculate the midpoint of the new dimensions
    midr = [Lr[j] / 2 for j in range(2)]
    
    # Initialize the rotated heightmap with zero values
    Zr = flat_height(Lr)
    
    for pos in Zr:
        # Calculate the coordinates relative to the center of the new heightmap
        d = [pos[j] - midr[j] for j in range(2)]
        
        # Rotate the coordinates by theta radians around the center
        x = int(d[0] * np.cos(theta * np.pi) + d[1] * np.sin(theta * np.pi) + mid[0])
        y = int(-d[0] * np.sin(theta * np.pi) + d[1] * np.cos(theta * np.pi) + mid[1])
        
        # Update the rotated heightmap with values from the original heightmap
        if (x, y) in Z:
            Zr[pos] = Z[x, y]
        else:
            Zr[pos] = 0  # Set to 0 if the position is out of bounds in the original heightmap
    
    return Zr

# Plot the rotated heightmap with a zoom level of 5
plot_height(rotate_height(Z, 0.25), terrain=None, zoom=5)


In [ ]:
start = time.time()  # Record the start time of the sample generation process

samples = 300  # Number of samples to generate
tartans = []  # List to store the generated tartans

# Generate the specified number of tartans
for j in range(samples):
    # Shuffle the heightmap Z according to the grid and get the shuffled heightmap randZ
    randZ, _ = shuffle_height(Z, grid)
    
    # Apply a random rotation to the shuffled heightmap
    randZ = rotate_height(randZ, random.random())
    
    # Add the rotated heightmap to the list of tartans
    tartans.append(randZ)

end = time.time()  # Record the end time of the sample generation process

# Print the time taken to generate the samples
print('Generation of', samples, 'samples took', int(end-start), 'seconds')


In [ ]:
# Define the size of the reduced grid
reduced_size = [10,10]

# Create a set of interactive checkbox widgets for the reduced grid
# The checkboxes are initially set to 'True' by default
peak_box = get_boxes(reduced_size)

# Display the checkboxes in a grid layout
# Each row (HBox) contains checkboxes for one row of the grid
# All rows (VBox) are stacked vertically to form the complete grid layout
VBox([ HBox([ peak_box[x,y] for x in range(reduced_size[0]) ]) for y in range(reduced_size[1]) ])


In [ ]:
# Create a set of interactive checkbox widgets for the reduced grid
# The checkboxes are initially set to 'False' by default
valley_box = get_boxes(reduced_size, value=False)

# Display the checkboxes in a grid layout
# Each row (HBox) contains checkboxes for one row of the grid
# All rows (VBox) are stacked vertically to form the complete grid layout
VBox([ HBox([ valley_box[x,y] for x in range(reduced_size[0]) ]) for y in range(reduced_size[1]) ])


In [ ]:
# Initialize an empty dictionary to store the height values
Zs = {}

# Iterate over each position in the reduced grid
for y in range(reduced_size[1]):
    for x in range(reduced_size[0]):
        # Check the value of the corresponding checkbox in the peak_box
        if peak_box[x, y].value == False:
            # Set height to 1 if the peak checkbox is unchecked
            Zs[x, y] = 1
        # Check the value of the corresponding checkbox in the valley_box
        elif valley_box[x, y].value == True:
            # Set height to 0 if the valley checkbox is checked
            Zs[x, y] = 0
        else:
            # Set height to 0.5 for all other cases
            Zs[x, y] = 0.5

# Plot the heightmap using the specified terrain colors and zoom level
plot_height(Zs, terrain=None, zoom=10)


In [ ]:
def shuffle_grid(grid):
    # Determine the number of bits based on the length of the grid
    n = int(np.log(len(grid)) / np.log(2))
    
    # Create a list of bit positions and shuffle it
    order = [j for j in range(n)]
    random.shuffle(order)
    
    # Create a new grid with shuffled bit positions
    new_grid = {}
    for pos in grid:
        new_string = ''
        for j in order:
            new_string = grid[pos][j] + new_string
        new_grid[pos] = new_string
    
    return new_grid

def shuffle_height(Z, grid):
    # Shuffle the grid
    new_grid = shuffle_grid(grid)
    
    # Create a new height map with shuffled positions
    new_Z = {}
    for pos in Z:
        # Find the new position for the current grid entry
        string = grid[pos] 
        new_pos = list(new_grid.keys())[list(new_grid.values()).index(string)]
        new_Z[new_pos] = Z[pos]
        
    return new_Z, new_grid

# Shuffle the height map and grid
Z, grid = shuffle_height(Z, grid)

# Plot the shuffled height map
plot_height(Z, terrain=None, zoom=5)


In [ ]:
def blur(Zs, reduced_size, steps=2):
    # Apply a blurring effect to the height map Zs
    for j in range(steps):
        for offset in [0, 1]:
            for y in range(1, reduced_size[1] - 1):
                # Iterate over every pixel, adjusting with a step size of 2
                for x in range(1 + (offset + y) % 2, reduced_size[0] - 1 + (offset + y) % 2, 2):
                    # Compute the average of the current pixel and its four neighbors
                    Zs[x, y] = (Zs[x, y] + (Zs[x + 1, y] + Zs[x - 1, y] + Zs[x, y + 1] + Zs[x, y - 1]) / 4) / 2
    return Zs

# Apply the blur function to the height map with 1 iteration
Zs = blur(Zs, reduced_size, steps=1)

# Plot the blurred height map
plot_height(Zs, terrain=None, zoom=10)


In [ ]:
def islands(size, Zs, tartans):
    # Create a height map by combining quantum tartans with basic map features

    # Initialize height map with flat height
    Z = flat_height(size)
        
    # Get the size of the first tartan for positioning
    tsize = max(tartans[0])
    
    for tartan in tartans:
        unchosen = True
        while unchosen:
            # Randomly select a starting position within the height map
            x0 = random.choice(range(size[0]))
            y0 = random.choice(range(size[1]))

            # Check if the selected position meets the condition based on Zs
            if random.random() < Zs[int(x0 * (max(Zs.keys())[0] + 1) / size[0]), int(y0 * (max(Zs.keys())[1] + 1) / size[1])]:
                unchosen = False

        # Place the tartan at the chosen position on the height map
        for (x, y) in tartan:
            xx = x - int(tsize[0] / 2) + x0
            yy = y - int(tsize[1] / 2) + y0
            if (xx, yy) in Z:
                Z[xx, yy] += tartan[x, y]
                
    # Normalize the height map values
    Z = normalize_height(Z)

    return Z


In [ ]:
# Define the size of the height map
size = [200, 200]

# Generate the height map by combining quantum tartans with basic map features
Z_islands = islands(size, Zs, tartans)

# Plot the generated height map with specified terrain thresholds
# The terrain argument defines the height ranges for different terrain types
plot_height(Z_islands, terrain=[2/16, 3/16, 5/16, 10/16, 12/16])


### The code to transform our newly generated terrain into terrain in Minecraft. To run the "Blocks" file, you have to download Minecraft Pi

In [ ]:
def make_blocks(Z, terrain=[2/16,3/16,5/16,10/16,12/16], height=24, depth=12):
    # Create a 3D map of blocks from the height map Z, defining different terrain features
    # terrain defines the height thresholds for different block types
    # height and depth control the vertical scale of the blocks

    def addBlocks(blocks, x1, h1, y1, x2, h2, y2, block):
        # Add blocks of a given type for a specified range of coordinates
        for x in range(x1, x2 + 1):
            for y in range(y1, y2 + 1):
                for h in range(h1, h2 + 1):
                    blocks[x, h, y] = block

    def addTreeBlocks(blocks, x, h, y, rnd):
        # Create a tree at the specified position
        for j in range(1, 6):
            blocks[x, h + j, y] = 'tree'
        for xx in range(x - 3, x + 4):
            for yy in range(y - 3, y + 4):
                for hh in range(h + 5, h + 11):
                    d = (xx - x) ** 2 + (yy - y) ** 2 + (hh - h - 6) ** 2 + 0.1
                    if d < 8:
                        blocks[xx, hh, yy] = 'leaves'
        xx = choose([x - 1, x + 1], rnd)
        yy = choose([y - 1, y + 1], rnd)
        blocks[xx, h + 5, yy] = 'tree'
        blocks[xx, h + 4, yy] = 'torch'

    def choose(options, rnd):
        # Randomly choose an option from a list based on a random number
        return options[int(round(rnd * (len(options) - 1)))]

    # Define the sea level based on the depth and terrain
    sea_level = int(depth + terrain[0] * height + 1)
    
    # Find a starting position for the blocks
    choosing = True
    while choosing:
        spawn = random.choice(list(Z.keys()))
        if Z[spawn] > terrain[0]:
            choosing = False
    spawn = [spawn[0], depth + height, spawn[1]]
    
    blocks = {}
    (Xmin, Hmin, Ymin) = (0, 0, 0)
    (Xmax, Hmax, Ymax) = (0, 0, 0)
    
    # Iterate through the height map to populate blocks
    for (X, Y) in Z:
        Hfloat = depth + Z[X, Y] * height
        H = int(Hfloat)  # Height for a block
        rnd = Hfloat - H  # Value from 0 to 1 used for randomness
        
        Xmin = min(Xmin, X)
        Ymin = min(Ymin, Y)
        Hmin = min(Hmin, H)
        Xmax = max(Xmax, X)
        Ymax = max(Ymax, Y)
        Hmax = max(Hmax, H)
        
        # Define mineral and block types based on height and terrain
        Hm = int((1 - Z[X, Y]) * depth / 2)  # Height for stalactites
        Ht = int(depth - (1 - Z[X, Y]) * depth / 2)  # Height at which stalagmites begin
        
        if Z[X, Y] < terrain[0]:
            minerals = ['diamondblock', 'goldblock']  # Precious minerals in hard-to-reach places
        else:
            minerals = ['stone', 'stone', 'stone_with_coal', 'stone_with_iron', 'stone_with_copper', 'stone_with_tin', 'stone_with_gold', 'stone_with_diamond']
        
        stone_m = choose(minerals, rnd)
        stone_t = choose(minerals, 1 - rnd)
        
        # Add blocks based on height thresholds
        if (1 - Z[X, Y]) < terrain[0]:  # Bottom of the cavern with lava
            blocks[X, 0, Y] = stone_m
            blocks[X, 1, Y] = 'lava_source'
        else:  # Add mineral blocks
            addBlocks(blocks, X, 0, Y, X, Hm, Y, stone_m)
        
        if Z[X, Y] < terrain[4]:  # Roof is always a mineral
            addBlocks(blocks, X, Ht, Y, X, depth, Y, stone_t)
        
        if rnd < 0.005 and Z[X, Y] > terrain[0] and Z[X, Y] < terrain[4]:
            blocks[X, Ht - 1, Y] = 'torch'
        
        # Define terrain features based on height
        if Z[X, Y] < terrain[0]:  # Sand and water for low terrain
            addBlocks(blocks, X, depth, Y, X, H, Y, 'sand')
            addBlocks(blocks, X, H + 1, Y, X, sea_level, Y, 'water_source')
        elif Z[X, Y] < terrain[1]:  # Sand
            addBlocks(blocks, X, depth + 1, Y, X, H - 1, Y, 'stone')
            blocks[X, H, Y] = 'sand'
            blocks[X, H + 1, Y] = 'sand'
        elif Z[X, Y] < terrain[2]:  # Grass with trees
            addBlocks(blocks, X, depth + 1, Y, X, H - 1, Y, 'stone')
            blocks[X, H, Y] = 'dirt_with_grass'
            blocks[X, H + 1, Y] = 'dirt_with_grass'
            if rnd < 0.025:
                addTreeBlocks(blocks, X, H, Y, rnd)
            else:
                blocks[X, H + 2, Y] = choose(['fern_1', 'marram_grass_1', 'marram_grass_2', 'marram_grass_3'], rnd)
        elif Z[X, Y] < terrain[3]:  # Grass with ferns
            addBlocks(blocks, X, depth + 1, Y, X, H - 1, Y, 'stone')
            blocks[X, H, Y] = 'dirt_with_grass'
            blocks[X, H + 1, Y] = 'dirt_with_grass'
            blocks[X, H + 2, Y] = choose(['fern_1', 'fern_2', 'fern_3', 'marram_grass_1'], rnd)
        elif Z[X, Y] < terrain[4]:  # Mixture of grass and stone
            addBlocks(blocks, X, depth + 1, Y, X, H - 1, Y, 'stone')
            if rnd < 1 / 3:
                blocks[X, H, Y] = 'dirt_with_grass'
                blocks[X, H + 1, Y] = 'dirt_with_grass'
            else:
                blocks[X, H, Y] = 'stone'
                blocks[X, H + 1, Y] = 'stone'
        elif H == (depth + terrain[4] * height):  # Just stone with additional height
            H += int(height * rnd / 10)
            addBlocks(blocks, X, depth + 1, Y, X, H + 1, Y, 'stone')
        elif Z[X, Y] == 1:  # Place a torch
            blocks[X, H, Y] = 'torch'
    
    # Return the blocks and coordinate bounds
    mins = (Xmin, Hmin, Ymin)
    maxs = (Xmax, Hmax, Ymax)
    
    return blocks, spawn, mins, maxs

def save_blocks(blocks, spawn, mins, maxs, filename='blocks.csv'):
    # Save the dictionary of blocks to a CSV file
    with open(filename, 'w') as file:
        file.write(str(mins[0]) + ',' + str(mins[1]) + ',' + str(mins[2]) + ',min,\n')
        file.write(str(maxs[0]) + ',' + str(maxs[1]) + ',' + str(maxs[2]) + ',max,\n')
        file.write(str(spawn[0]) + ',' + str(spawn[1]) + ',' + str(spawn[2]) + ',player,\n')
        for (x, h, y) in blocks:
            file.write(str(x) + ',' + str(h) + ',' + str(y) + ',' + blocks[x, h, y] + ',\n')


In [ ]:
# Generate blocks for the 3D terrain based on the height map and save the result
blocks, spawn, mins, maxs = make_blocks(Z_islands)

# Save the generated blocks, spawn point, and coordinate bounds to a CSV file
save_blocks(blocks, spawn, mins, maxs)
